In [1]:
import datetime
from ib_async import *
import pandas as pd
from arcticdb import Arctic, QueryBuilder, LibraryOptions

import sys
from pathlib import Path

# Get the parent directory of the current notebook's directory
project_root = Path.cwd().parent

# Add the project root to the system path
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

try:
    from strategy_manager.strategies.short_vix import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from data_and_research import ac, initialize_db
except:
    from strategy_manager.strategies.short_vix import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from data_and_research import ac, initialize_db

In [2]:
ac = initialize_db('db')


In [3]:
ac.list_libraries()

['general', 'pnl', 'portfolio', 'spx500', 'stocks']

In [4]:
import yfinance as yf

In [9]:
dfa = yf.download("A",start="2022-01-06",end="2022-01-07")

[*********************100%%**********************]  1 of 1 completed


In [13]:
dfa['Symbol'] = "A"
dfa = dfa[['Symbol','Open','High','Low','Close','Volume']] 


In [15]:
dfa

,Symbol,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2022-01-06,A,148.850006,149.960007,145.580002,149.119995,146.566086,2298300


In [16]:
lib = ac.get_library('spx500')
df = lib.read('ALL_STOCKS').data

In [58]:
df.head()
symbols = df[:5]['Symbol'].to_list()
symbols

['A', 'EMN', 'EMR', 'ENPH', 'EOG']

In [67]:
dfa = yf.download(symbols,start="2022-01-06",end="2022-01-08",group_by="Ticker")
dfa

[*********************100%%**********************]  5 of 5 completed


Ticker            ENPH                                                  \
Price             Open        High         Low       Close   Adj Close   
Date                                                                     
2022-01-06  156.009995  156.229996  145.820007  151.490005  151.490005   
2022-01-07  152.110001  154.949997  144.169998  145.130005  145.130005   

Ticker                        A                                      ...  \
Price        Volume        Open        High         Low       Close  ...   
Date                                                                 ...   
2022-01-06  3546200  148.850006  149.960007  145.580002  149.119995  ...   
2022-01-07  3563000  149.119995  149.729996  145.089996  145.149994  ...   

Ticker            EMR                                        EMN              \
Price             Low      Close  Adj Close   Volume        Open        High   
Date                                                                           
2022-01-06  95.269997  96.230003  90.501770  2477000  123.029999  123.639999   
2022-01-07  95.610001  95.690002  89.993896  2863600  122.339996  123.190002   

Ticker                                                   
Price              Low       Close   Adj Close   Volume  
Date                                                     
2022-01-06  121.190002  122.150002  111.760788  1034600  
2022-01-07  121.839996  122.410004  111.998688   970500  

[2 rows x 30 columns]

In [69]:
dfa = dfa.stack(level=0).rename_axis(['Date', 'Symbol'])[['Open','High','Low','Close','Volume']]#.reset_index(level=1)

In [71]:
df_neu = df.head(5)
df_neu = df_neu.iloc[:,:6]
df_neu

,Symbol,Open,High,Low,Close,Volume
Date,,,,,,
2022-01-06,A,146.810340,147.905131,143.585144,147.076630,2298300.0
2022-01-06,EMN,114.486667,115.054309,112.774442,113.667778,1034600.0
2022-01-06,EMR,91.525911,92.461111,90.915159,91.831284,2477000.0
2022-01-06,ENPH,156.009995,156.229996,145.820007,151.490005,3546200.0
2022-01-06,EOG,86.148351,86.281229,83.774288,84.615837,5254700.0


,Symbol,Open,High,Low,Close,Volume
Date,,,,,,
2022-01-06,A,146.810340,147.905131,143.585144,147.076630,2298300.0
2022-01-06,EMN,114.486667,115.054309,112.774442,113.667778,1034600.0
2022-01-06,EMR,91.525911,92.461111,90.915159,91.831284,2477000.0
2022-01-06,ENPH,156.009995,156.229996,145.820007,151.490005,3546200.0
2022-01-06,EOG,86.148351,86.281229,83.774288,84.615837,5254700.0


In [44]:
nlib = ac.get_library('stocks')
nlib.write('test',df_neu)

VersionedItem(symbol='test', library='stocks', data=n/a, version=2, metadata=None, host='S3(endpoint=s3.eu-central-1.amazonaws.com, bucket=lowquant-arcticdb)')

In [45]:
nlib.read('test').data

,Symbol,Open,High,Low,Close,Volume
Date,,,,,,
2022-01-06,A,146.810340,147.905131,143.585144,147.076630,2298300.0
2022-01-06,EMN,114.486667,115.054309,112.774442,113.667778,1034600.0
2022-01-06,EMR,91.525911,92.461111,90.915159,91.831284,2477000.0
2022-01-06,ENPH,156.009995,156.229996,145.820007,151.490005,3546200.0
2022-01-06,EOG,86.148351,86.281229,83.774288,84.615837,5254700.0


In [40]:
# nlib.update('test',dfa)
# dfa['Volume'] = dfa['Volume'].astype(float)
# dfa

nlib.update('test',dfa)

VersionedItem(symbol='test', library='stocks', data=n/a, version=1, metadata=None, host='S3(endpoint=s3.eu-central-1.amazonaws.com, bucket=lowquant-arcticdb)')

In [41]:
nlib.read('test').data

,Symbol,Open,High,Low,Close,Volume
Date,,,,,,
2022-01-06,A,148.850006,149.960007,145.580002,149.119995,2298300.0


In [72]:
# Store df_neu as Multi Index
df_ac_multi = df_neu.set_index('Symbol', append=True)
df_ac_multi

Open        High         Low       Close     Volume
Date       Symbol                                                           
2022-01-06 A       146.810340  147.905131  143.585144  147.076630  2298300.0
           EMN     114.486667  115.054309  112.774442  113.667778  1034600.0
           EMR      91.525911   92.461111   90.915159   91.831284  2477000.0
           ENPH    156.009995  156.229996  145.820007  151.490005  3546200.0
           EOG      86.148351   86.281229   83.774288   84.615837  5254700.0

In [73]:
dfa

Price                    Open        High         Low       Close   Volume
Date       Symbol                                                         
2022-01-06 A       148.850006  149.960007  145.580002  149.119995  2298300
           EMN     123.029999  123.639999  121.190002  122.150002  1034600
           EMR      95.910004   96.889999   95.269997   96.230003  2477000
           ENPH    156.009995  156.229996  145.820007  151.490005  3546200
           EOG      97.250000   97.400002   94.570000   95.519997  5254700
2022-01-07 A       149.119995  149.729996  145.089996  145.149994  2058600
           EMN     122.339996  123.190002  121.839996  122.410004   970500
           EMR      96.550003   96.970001   95.610001   95.690002  2863600
           ENPH    152.110001  154.949997  144.169998  145.130005  3563000
           EOG      96.010002   98.269997   94.889999   98.160004  6277800

In [85]:
# lib.write('multi', df_ac_multi)
lib.update('multi',df_ac_multi,upsert=False)

VersionedItem(symbol='multi', library='spx500', data=n/a, version=6, metadata=None, host='S3(endpoint=s3.eu-central-1.amazonaws.com, bucket=lowquant-arcticdb)')

In [88]:
df = lib.read('multi').data

In [77]:
dfa['Volume'] = dfa['Volume'].astype(float)

In [83]:
lib.update('multi',dfa,upsert=False)

VersionedItem(symbol='multi', library='spx500', data=n/a, version=5, metadata=None, host='S3(endpoint=s3.eu-central-1.amazonaws.com, bucket=lowquant-arcticdb)')

In [87]:
df = lib.read('multi').data

In [91]:

    # df = data.stack(level=0).rename_axis(['Date', 'Symbol']).reset_index(level=1)
    # df = df.sort_values(by='Symbol',axis='index',kind='stable')
    
df.reset_index(level=1).sort_values(by='Symbol',axis='index',kind='stable')

,Symbol,Open,High,Low,Close,Volume
Date,,,,,,
2022-01-06,A,146.810340,147.905131,143.585144,147.076630,2298300.0
2022-01-07,A,149.119995,149.729996,145.089996,145.149994,2058600.0
2022-01-06,EMN,114.486667,115.054309,112.774442,113.667778,1034600.0
2022-01-07,EMN,122.339996,123.190002,121.839996,122.410004,970500.0
2022-01-06,EMR,91.525911,92.461111,90.915159,91.831284,2477000.0
2022-01-07,EMR,96.550003,96.970001,95.610001,95.690002,2863600.0
2022-01-06,ENPH,156.009995,156.229996,145.820007,151.490005,3546200.0
2022-01-07,ENPH,152.110001,154.949997,144.169998,145.130005,3563000.0
2022-01-06,EOG,86.148351,86.281229,83.774288,84.615837,5254700.0


In [93]:
q = QueryBuilder()
q = q.date_range((pd.Timestamp("2022-01-07"), pd.Timestamp("2024-08-08")))
lib.read('multi',query_builder=q).data

Open        High         Low       Close     Volume
Date       Symbol                                                           
2022-01-07 A       149.119995  149.729996  145.089996  145.149994  2058600.0
           EMN     122.339996  123.190002  121.839996  122.410004   970500.0
           EMR      96.550003   96.970001   95.610001   95.690002  2863600.0
           ENPH    152.110001  154.949997  144.169998  145.130005  3563000.0
           EOG      96.010002   98.269997   94.889999   98.160004  6277800.0

In [5]:
lib = ac.get_library('portfolio')
lib.list_symbols()

['U7706434',
 'test2',
 'DU3180649_AAPL',
 'U7706434_SMAG7',
 'DU3180649',
 'positions',
 'test3',
 'test']

In [7]:
lib.read('test3').data

,timestamp,symbol,asset class,position,% of nav,averageCost,marketPrice,pnl %,strategy,contract,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,2024-07-24 11:03,AAPL,STK,37.0,3.647876,180.260541,223.979996,24.253481,,"Stock(conId=265598, symbol='AAPL', right='0', ...",...,,False,,8287.260000,1617.620000,USD,0.0,DU3180649,7648.001772,1.083585
1,2024-07-24 11:03,CNR,STK,-699.0,-36.924325,174.287153,165.533997,5.022261,,"Stock(conId=4458894, symbol='CNR', right='0', ...",...,,False,,-115708.260000,6118.460000,CAD,0.0,DU3180649,-77414.176421,1.494665
2,2024-07-24 11:03,SIE,STK,1.0,0.082425,181.640000,172.812302,-4.859997,,"Stock(conId=14217, symbol='SIE', right='0', pr...",...,,False,,172.810000,-8.830000,EUR,0.0,DU3180649,172.810000,1.000000
3,2024-07-24 11:03,VOW,STK,-176.0,-9.306355,141.129400,110.860001,21.447976,,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,-19511.360000,5327.410000,EUR,0.0,DU3180649,-19511.360000,1.000000
4,2024-07-24 11:05,AAPL,STK,37.0,3.647876,180.260541,223.979996,24.253481,,"Stock(conId=265598, symbol='AAPL', right='0', ...",...,,False,,8287.259843,43.719455,USD,0.0,DU3180649,7648.001772,1.083585
5,2024-07-24 11:05,CNR,STK,-699.0,-36.924325,174.287153,165.533997,5.022261,,"Stock(conId=4458894, symbol='CNR', right='0', ...",...,,False,,-115708.263623,-8.753157,CAD,0.0,DU3180649,-77414.176421,1.494665
6,2024-07-24 11:05,SIE,STK,1.0,0.082425,181.640000,172.812302,-4.859997,,"Stock(conId=14217, symbol='SIE', right='0', pr...",...,,False,,172.812302,-8.827698,EUR,0.0,DU3180649,172.810000,1.000000
7,2024-07-24 11:05,VOW,STK,-176.0,-9.306355,141.129400,110.860001,21.447976,,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,-19511.360106,-30.269399,EUR,0.0,DU3180649,-19511.360000,1.000000
8,2024-07-24 11:21,AAPL,STK,37.0,3.647607,180.260541,224.490005,24.536410,,"Stock(conId=265598, symbol='AAPL', right='0', ...",...,,False,,8306.130203,44.229465,USD,0.0,DU3180649,7648.001627,1.083585
9,2024-07-24 11:21,CNR,STK,-699.0,-36.921605,174.287153,165.533997,5.022261,,"Stock(conId=4458894, symbol='CNR', right='0', ...",...,,False,,-115708.263623,-8.753157,CAD,0.0,DU3180649,-77414.178845,1.494665
